In [12]:
from dataset_processing import question_and_answer_pairs_from_log_file, remove_quotes_from_file, filter_by_max_words, clean_text
import pandas as pd
import nltk
from nltk.corpus import stopwords
import os
import platform

    #This notebook is used to train the ANNABELL model on the SQuAD dataset.
It uses the declarative statements generated from the SQuAD dataset to train the model in specialist subjects.
The steps to rain the model are as follows:
1) prepare the dataset
2) pre-train the annabell model by loading pretrained weights form the people/body/skills file
3) train the model on the declarative statements
4) load the weights

In [18]:
operating_system = platform.system()
if operating_system == 'Windows':
    raise Exception("not used on Windows yet")
elif operating_system == 'Linux':
    test_input_dir = "/run/user/1000/gvfs/google-drive:host=gmail.com,user=cjameswalmsley/0AG4X1_cOZQRnUk9PVA/0B24X1_cOZQRnVjZEemxXd3BNSTQ/1SSfZglixkSB5aj0qMv2BsDojVhxNOv0o/11BZhonitY-Jfw5xqM1xAl5TjJPRDRKHr/1OyCmWPf_7PH1bCc-5GLvBdo-gvEvJsiP/1dgL8C9ydesEz3jM28e55yGR9PECRtjaA/1_z0x1Ym8ceREizMlb1mk29msDhvfy2xg/19QWr_JOiOoob5pcpd9aZHJ5kzB4Q0Dc-/1Coru7UDS-tRqhb3jkpMEHM2UyG-hMao_/1DBHxcQpfXReIx0FZn9LRR_aS7VZUuf2V"
    test_log_dir = "/run/user/1000/gvfs/google-drive:host=gmail.com,user=cjameswalmsley/0AG4X1_cOZQRnUk9PVA/0B24X1_cOZQRnVjZEemxXd3BNSTQ/1SSfZglixkSB5aj0qMv2BsDojVhxNOv0o/11BZhonitY-Jfw5xqM1xAl5TjJPRDRKHr/1OyCmWPf_7PH1bCc-5GLvBdo-gvEvJsiP/1dgL8C9ydesEz3jM28e55yGR9PECRtjaA/1_z0x1Ym8ceREizMlb1mk29msDhvfy2xg/19QWr_JOiOoob5pcpd9aZHJ5kzB4Q0Dc-/1Coru7UDS-tRqhb3jkpMEHM2UyG-hMao_/1DZfo5MVzCEfoiO3ulkbx6FMAo8e5o_SX"
    train_dir = "/run/user/1000/gvfs/google-drive:host=gmail.com,user=cjameswalmsley/0AG4X1_cOZQRnUk9PVA/0B24X1_cOZQRnVjZEemxXd3BNSTQ/1SSfZglixkSB5aj0qMv2BsDojVhxNOv0o/11BZhonitY-Jfw5xqM1xAl5TjJPRDRKHr/1OyCmWPf_7PH1bCc-5GLvBdo-gvEvJsiP/1dgL8C9ydesEz3jM28e55yGR9PECRtjaA/1_z0x1Ym8ceREizMlb1mk29msDhvfy2xg/1951JmEhJWAQjilgGc7T00-z9bbiJwNM3"
elif operating_system == 'Darwin': #macOS
    test_log_dir = "/Users/chris/PycharmProjects/dataset/docker/docker_train/shared_data/statements/logs"
    test_input_dir = "/Users/chris/PycharmProjects/dataset/docker/docker_train/shared_data/statements/testing"
    train_dir = "/Users/chris/Library/CloudStorage/GoogleDrive-cjameswalmsley@gmail.com/My Drive/Shared with Julia/Education/Kent University/PhD/work/annabell/training"
else:
    raise Exception("unsupported OS")

test_log_filename = "logfile_squad_test_nyc.txt"
test_input_filename = "test_nyc_questions.txt"
train_filename = "declarative_sentences_train_gemma3:4b_20250617_201822.tsv"

test_log_filepath = os.path.join(test_log_dir, test_log_filename)
test_input_filepath = os.path.join(test_input_dir, test_input_filename)
train_filepath = os.path.join(train_dir, train_filename)

In [19]:
question_and_answer_pairs = question_and_answer_pairs_from_log_file(test_log_filepath)

#print("length of log file questions: " + str(len(questions)))
#print("length of log file answers: " + str(len(answers)))
print("length of log file questions and answers: " + str(len(question_and_answer_pairs)))

with open(test_input_filepath, 'r') as test_input_file:
    test_input_lines = test_input_file.readlines()
print("length of test file questions: " + str(len(test_input_lines)))

length of log file questions and answers: 586
length of log file questions and answers: 586
length of test file questions: 1612


In [20]:
#Load and clean the training data derived from the SQuAD dataset
train_filepath = remove_quotes_from_file(train_filepath)
train_df = pd.read_csv(train_filepath, sep="\t")
train_df = train_df.dropna()

filtered_train_df = train_df[train_df["title"] == "New_York_City"]
filtered_train_df = filter_by_max_words(filtered_train_df, max_words=20)
filtered_train_df["response_question"] = clean_text(filtered_train_df["response_question"], True)
filtered_train_df["response_answer"] = clean_text(filtered_train_df["response_answer"], False)
filtered_train_df["statement"] = clean_text(filtered_train_df["statement"], False)
filtered_train_df.reset_index(drop=True, inplace=True)
filtered_train_df

Cleaned data saved to /run/user/1000/gvfs/google-drive:host=gmail.com,user=cjameswalmsley/0AG4X1_cOZQRnUk9PVA/0B24X1_cOZQRnVjZEemxXd3BNSTQ/1SSfZglixkSB5aj0qMv2BsDojVhxNOv0o/11BZhonitY-Jfw5xqM1xAl5TjJPRDRKHr/1OyCmWPf_7PH1bCc-5GLvBdo-gvEvJsiP/1dgL8C9ydesEz3jM28e55yGR9PECRtjaA/1_z0x1Ym8ceREizMlb1mk29msDhvfy2xg/1951JmEhJWAQjilgGc7T00-z9bbiJwNM3/declarative_sentences_train_gemma3:4b_20250617_201822_cleaned.tsv


,id,title,question,answer,response_question,response_answer,statement
0,56ce304daab44d1400b8850e,New_York_City,What city in the United States has the highest...,New York,? What city in the United States has the highe...,new york,new york city has the highest population in th...
1,56ce304daab44d1400b8850f,New_York_City,In what city is the United Nations based?,New York,? in what city is the united nations based,new york,the united nations is based in New York
2,56ce304daab44d1400b88510,New_York_City,What city has been called the cultural capital...,New York,? what city has been called the cultural capit...,new york,new york has been called the cultural capital ...
3,56ce304daab44d1400b88511,New_York_City,What American city welcomes the largest number...,New York,? What American city welcomes the largest numb...,new york,new york welcomes the largest number of legal ...
4,56cf5d41aab44d1400b89130,New_York_City,The major gateway for immigration has been whi...,New York City,? the major gateway for immigration has been w...,new york city,the major gateway for immigration has been in ...
...,...,...,...,...,...,...,...
801,56d1218c17492d1400aaba1e,New_York_City,What ZIP code was responsible for the greatest...,10021,? what ZIP code was responsible for the greate...,10021,the ZIP code 10021 was responsible for the gre...
802,56d1218c17492d1400aaba1f,New_York_City,How much money in cents does New York City rec...,83,? how much money in cents does New York City r...,83,New York City receives 83 cents for every doll...
803,56d1218c17492d1400aaba20,New_York_City,How much more money does the city give to the ...,$11 billion,? how much more money does the city give to th...,11 billion,the city gives 11 billion to the state of new ...
804,56d1218c17492d1400aaba21,New_York_City,"Each year, how much more money does New York C...",$11.4 billion,? each year how much more money does New York ...,11 point 4 billion,new york city gives 11 point 4 billion more to...


In [21]:
#add the test questions to the dataframe
for question, answer in question_and_answer_pairs:
    if question in filtered_train_df["response_question"].values:
        filtered_train_df.loc[filtered_train_df["response_question"] == question, "test_answer"] = answer
    else:
        print(f"Question not found in training data: {question}")
filtered_train_df

Question not found in training data: ? what was the myth that Manhattan was bought for by General Peter Minuit
Question not found in training data: ? which in


,id,title,question,answer,response_question,response_answer,statement,test_answer
0,56ce304daab44d1400b8850e,New_York_City,What city in the United States has the highest...,New York,? What city in the United States has the highe...,new york,new york city has the highest population in th...,the us
1,56ce304daab44d1400b8850f,New_York_City,In what city is the United Nations based?,New York,? in what city is the united nations based,new york,the united nations is based in New York,Exploitation number of updates >= 4000
2,56ce304daab44d1400b88510,New_York_City,What city has been called the cultural capital...,New York,? what city has been called the cultural capit...,new york,new york has been called the cultural capital ...,... Exploitation number of updates >= 4000
3,56ce304daab44d1400b88511,New_York_City,What American city welcomes the largest number...,New York,? What American city welcomes the largest numb...,new york,new york welcomes the largest number of legal ...,
4,56cf5d41aab44d1400b89130,New_York_City,The major gateway for immigration has been whi...,New York City,? the major gateway for immigration has been w...,new york city,the major gateway for immigration has been in ...,
...,...,...,...,...,...,...,...,...
801,56d1218c17492d1400aaba1e,New_York_City,What ZIP code was responsible for the greatest...,10021,? what ZIP code was responsible for the greate...,10021,the ZIP code 10021 was responsible for the gre...,NaN
802,56d1218c17492d1400aaba1f,New_York_City,How much money in cents does New York City rec...,83,? how much money in cents does New York City r...,83,New York City receives 83 cents for every doll...,NaN
803,56d1218c17492d1400aaba20,New_York_City,How much more money does the city give to the ...,$11 billion,? how much more money does the city give to th...,11 billion,the city gives 11 billion to the state of new ...,NaN
804,56d1218c17492d1400aaba21,New_York_City,"Each year, how much more money does New York C...",$11.4 billion,? each year how much more money does New York ...,11 point 4 billion,new york city gives 11 point 4 billion more to...,NaN


In [22]:
#remove rows where the test answer is longer than 20 words
filtered_train_df.dropna(subset=["test_answer"], inplace=True)
filtered_train_df = filtered_train_df[filtered_train_df.apply(lambda row: len(row["test_answer"].split()) <= 20 , axis=1)]
filtered_train_df.reset_index(drop=True, inplace=True)
filtered_train_df

,id,title,question,answer,response_question,response_answer,statement,test_answer
0,56ce304daab44d1400b8850e,New_York_City,What city in the United States has the highest...,New York,? What city in the United States has the highe...,new york,new york city has the highest population in th...,the us
1,56ce304daab44d1400b8850f,New_York_City,In what city is the United Nations based?,New York,? in what city is the united nations based,new york,the united nations is based in New York,Exploitation number of updates >= 4000
2,56ce304daab44d1400b88510,New_York_City,What city has been called the cultural capital...,New York,? what city has been called the cultural capit...,new york,new york has been called the cultural capital ...,... Exploitation number of updates >= 4000
3,56ce304daab44d1400b88511,New_York_City,What American city welcomes the largest number...,New York,? What American city welcomes the largest numb...,new york,new york welcomes the largest number of legal ...,
4,56cf5d41aab44d1400b89130,New_York_City,The major gateway for immigration has been whi...,New York City,? the major gateway for immigration has been w...,new york city,the major gateway for immigration has been in ...,
...,...,...,...,...,...,...,...,...
566,56d108f917492d1400aab7b6,New_York_City,What artistic style was prominent in New York ...,abstract expressionism,? what artistic style was prominent in New Yor...,abstract expressionism,abstract expressionism was prominent in New Yo...,...
567,56d108f917492d1400aab7b7,New_York_City,What was the name of the new musical style tha...,hip hop,? what was the name of the new musical style t...,hip hop,the new musical style that emerged from New Yo...,
568,56d000c5234ae51400d9c24d,New_York_City,The fashion capital of the world is what city ...,New York City,? the fashion capital of the world is what cit...,new york city,the fashion capital of the world is in new yor...,
569,56d10ab417492d1400aab7e8,New_York_City,"In 2012-3, what number of people saw a show on...",11.57 million,? in what year were the five borough -s combin...,11 point 57 million,the number of people who saw a show on broadwa...,


In [23]:
filtered_train_df["test_answer_correct"] = filtered_train_df["test_answer"] == filtered_train_df["response_answer"]
percentage_correct =  filtered_train_df["test_answer_correct"].mean() * 100
percentage_correct

/tmp/ipykernel_21255/3093508224.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train_df["test_answer_correct"] = filtered_train_df["test_answer"] == filtered_train_df["response_answer"]


np.float64(0.0)

In [24]:
#filtered_train_df.dropna(subset=["test_answer"], inplace=True)

def any_word_match(row):
    # Split both strings into sets of words for efficient lookup
    try:
        stopwords.words('english')
    except LookupError:
        nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    test_words = set(row["test_answer"].split())
    response_words = set(row["response_answer"].split())
    intersecting_words = test_words.intersection(response_words)
    open_class_intersecting_words = intersecting_words - stop_words

    return open_class_intersecting_words != set()

filtered_train_df["test_answer_any_matching_word"] = filtered_train_df.apply(any_word_match, axis=1)

percentage_any_word_matches =  filtered_train_df["test_answer_any_matching_word"].mean() * 100
print(f"percentage any word matches = {percentage_any_word_matches} %")

percentage any word matches = 0.3502626970227671 %


[nltk_data] Downloading package stopwords to /home/chris/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/tmp/ipykernel_21255/3207302339.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train_df["test_answer_any_matching_word"] = filtered_train_df.apply(any_word_match, axis=1)


In [25]:
any_matches = filtered_train_df[filtered_train_df["test_answer_any_matching_word"]]
any_matches

,id,title,question,answer,response_question,response_answer,statement,test_answer,test_answer_correct,test_answer_any_matching_word
353,56cf3629aab44d1400b88e65,New_York_City,What borough housed the first Chinatown in New...,Manhattan,? What borough housed the first Chinatown in N...,Manhattan,the first Chinatown in New York was housed in ...,the Manhattan,False,True
534,56d1057217492d1400aab75c,New_York_City,What does the acronym NYPD stand for?,New York City Police Department,? what does the acronym nypd stand for,New York City Police Department,the acronym nypd stands for New York City Poli...,York go -s to the school,False,True
